# MBAM model reduction for the MWC model of BK
Model taken from Aldrich 2002 (Equation 1) <br>
Initial parameters taken from Miranda 2013 PNAS, Suppl fig 5 <br>
Algorithm as in Transtrum 2015 (Equation 49-51) - except for signs

The code below iteratively finds limits (boundaries) and stiff nonlinear combinations of sloppy parameters for the MWC model of BK, while preserving the model output function $P_{open}(V, [Ca] \ | \ \theta)$.

The iterative model reduction proceeds as follows:
+ $\theta = \{L_0, z_L, J_0, z_J, K_d, C, D, E \}$
    * $z_L -> 0$
    
    
+ $\phi^1 = \{L_0, J_0, z_J, K_d, C, D, E \}$
    * $L_0^{1/4} \rightarrow 0, \quad C \rightarrow \infty, \quad D \rightarrow \infty, \quad E \rightarrow 0 $
    

+ $\phi^2 = \{J_0, z_J, K_d, L_0^{1/4}C, L_0^{1/4}D, L_0^{-1/4}E \}$ 
    * $L_0^{1/4}C \rightarrow 0, \quad L_0^{-1/4}E \rightarrow \infty $
    
    
+ $\phi^3 = \{J_0, z_J, K_d, CE, L_0^{1/4}D \}$ 
    * $J_0 \rightarrow 0, \quad L_0^{1/4}D \rightarrow \infty $
 
 
+ $\phi^4 = \{J_0L_0^{1/4}D, z_J, K_d, CE  \}$ 
    * $K_d \rightarrow \infty, \quad CE \rightarrow \infty $
 
 
+ $\phi^5 = \{J_0L_0^{1/4}D, z_J, CEK_d^{-1}  \}$ 

The $P_{open}(V, [Ca] \ | \ \phi^5)$ function is an excellent approximation of the original (way below experimental error), despite having only 3 parameters. Each of the parameters are meaningful, the first one describing the slope of the voltage sensitivity, the second the middle point of the voltage sensitivity and the third the calcium sensitivity.
Note that the 4 parameter-model ($\phi^4$) describes both the voltage and the calcium sensitivity curves as two-parameter sigmoids. The allosteric MWC model introduces nonlinear interaction between these two sensitivity curves.

In [ ]:
using NBInclude #Includes for ipynb-s
#using JLD # Julia data I/O
nbinclude("MBAM.ipynb")
nbinclude("BK_functions/BK_functions.ipynb")


In [ ]:
# Create grid to evaluate over
#=
using Iterators
Ca=([0.0, 0.7, 4.0, 12.0, 22.0, 55.0, 70.0, 95.0]*1e-6)
V=collect(-100:25:200)*1e-3
Ca = float(Ca)
V = float(V)

x_grid = Array(Any, (length(Ca)*length(V),))
iterind = 0;
for j1 in product(Ca, V)
    iterind += 1;
    x_grid[iterind] = collect(j1)
end
size(x_grid[1])
=#

In [ ]:
#=
# Single MBAM call that runs through all iterations of BK model reduction with NON-AUTOMATED model reduction
phi0 = [2.2e-6, 0.42, 0.1026, 0.58, 39*1e-6, 6.16,30.4,2.0];
outp = MBAM(BK_simulator, phi0, x_grid, model_iters=[10, 11, 12, 13, 14], 
move_dir=[-1, -1, 1, 1, -1], boundary_time=10, verbose=1, reduce_func=BK_reduce);
=#


# Fit the reduced models to data
Using data from Aldrich 2010 Fig 4c,d fit simplified models

In [ ]:
# Read the data
data_4c_control = readcsv("Figures/aldrich_fig4c_control.csv")[1:end-1,:]
data_4c_lrrc26 = readcsv("Figures/aldrich_fig4c_lrrc26.csv")
data_4d_control = readcsv("Figures/aldrich_fig4d_control.csv")
data_4d_lrrc26 = readcsv("Figures/aldrich_fig4d_lrrc26.csv");


# Models for simultaneously fitting all 4 curves

In [ ]:
# join the data
data_x = [data_4c_control[:,1]; data_4c_lrrc26[:,1]; data_4d_control[:,1]; data_4d_lrrc26[:,1]]*1e-3;
data_y = [data_4c_control[:,2]; data_4c_lrrc26[:,2]; log10(data_4d_control[:,2]); log10(data_4d_lrrc26[:,2])];
data_in_log = [zeros(length(data_4c_control[:,1]),1); zeros(length(data_4c_lrrc26[:,1]),1); 
    ones(length(data_4d_control[:,1]),1); ones(length(data_4d_lrrc26[:,1]),1)].==1;
dataset_ranges = Any[1:length(data_4c_control[:,1]),
    (length(data_4c_control[:,1])+1):(length(data_4c_control[:,1])+length(data_4c_lrrc26[:,1])),
    (length(data_4c_control[:,1])+length(data_4c_lrrc26[:,1])+1):(length(data_4c_control[:,1])+length(data_4c_lrrc26[:,1])+length(data_4d_control[:,1])),
    (length(data_4c_control[:,1])+length(data_4c_lrrc26[:,1])+length(data_4d_control[:,1])+1):length(data_x)]

# Tradeoff between log-space fit and original space fit by norm of data there
w_log = norm([data_4c_control[:,2]; data_4c_lrrc26[:,2]])./norm([data_4d_control[:,2]; data_4d_lrrc26[:,2]])

In [ ]:
# Return results in log
function to_log(x::AbstractArray, in_log::AbstractArray)
    x = copy(x);
    x[x.<1e-12]=1e-12
    x[find(a->a==true, in_log)] = log10(x[find(a->a==true, in_log)])
    return x
end

function scale_data(x::AbstractArray, to_scale::AbstractArray, w)
    x = copy(x);
    x[find(a->a==true, to_scale)] *= w;
    return x
end

## Models

In [ ]:
using LsqFit
using Physical

elec_resp = (Volt*(-1)*e_electron/(k_boltzmann*304*Kelvin));


function model_orig(x::AbstractArray, ϕ::AbstractArray)
    D = ϕ[1]; L0 = ϕ[2]; J0 = ϕ[3]; zJ = ϕ[4]; zL = ϕ[5];
    Popen = (1+
    (1.+J0.*exp(-zJ .* x .* elec_resp )).^4
    ./
    ( L0.*exp(-zL .* x .* elec_resp ).*(1+D.*J0.*exp(-zJ .* x .* elec_resp )).^4)
    ).^-1
    
    logPopen = copy(Popen);
    for p1 = 1:length(logPopen)
        logPopen[p1] = Popen[p1]>0?log(Popen[p1]):-6
    end
    return Popen
end

function model_red(x::AbstractArray, ϕ::AbstractArray)
    DL0 = ϕ[1]; J0 = ϕ[2]; zJ = ϕ[3]; zL = ϕ[4];
    Popen = (1+
    (1.+J0.*exp(-zJ .* x .* elec_resp )).^4
    ./
    (exp(-zL .* x .* elec_resp ).*(DL0.*J0.*exp(-zJ .* x .* elec_resp )).^4)
    ).^-1
    return Popen
end

function model_Dmoves(x::AbstractArray, ϕ::AbstractArray; in_log=(zeros(size(x)).==1), data_ranges=Any[collect(1:length(x))], w_log=1)
    y_out = zeros(size(x))
    y_out[data_ranges[1]] = model_orig(x[data_ranges[1]], [ϕ[1]; ϕ[2:5]])
    y_out[data_ranges[2]] = model_orig(x[data_ranges[2]], [ϕ[6]; ϕ[2:5]])
    y_out[data_ranges[3]] = model_orig(x[data_ranges[3]], [ϕ[1]; ϕ[2:5]])
    y_out[data_ranges[4]] = model_orig(x[data_ranges[4]], [ϕ[6]; ϕ[2:5]])
    return scale_data(to_log(y_out, in_log), in_log, w_log);
end


function model_LDmoves(x::AbstractArray, ϕ::AbstractArray; in_log=(zeros(size(x)).==1), data_ranges=Any[collect(1:length(x))], w_log=1)
    y_out = zeros(size(x))
    y_out[data_ranges[1]] = model_red(x[data_ranges[1]], [ϕ[1]; ϕ[2:4]])
    y_out[data_ranges[2]] = model_red(x[data_ranges[2]], [ϕ[5]; ϕ[2:4]])
    y_out[data_ranges[3]] = model_red(x[data_ranges[3]], [ϕ[1]; ϕ[2:4]])
    y_out[data_ranges[4]] = model_red(x[data_ranges[4]], [ϕ[5]; ϕ[2:4]])
    return scale_data(to_log(y_out, in_log), in_log, w_log);
end


function model_red_allmoves(x::AbstractArray, ϕ::AbstractArray; in_log=(zeros(size(x)).==1), data_ranges=Any[collect(1:length(x))], w_log=1)
    y_out = zeros(size(x))
    y_out[data_ranges[1]] = model_red(x[data_ranges[1]], [ϕ[1]; ϕ[2:4]])
    y_out[data_ranges[2]] = model_red(x[data_ranges[2]], [ϕ[5]; ϕ[6:8]])
    y_out[data_ranges[3]] = model_red(x[data_ranges[3]], [ϕ[1]; ϕ[2:4]])
    y_out[data_ranges[4]] = model_red(x[data_ranges[4]], [ϕ[5]; ϕ[6:8]])
    return scale_data(to_log(y_out, in_log), in_log, w_log);
end

In [ ]:
weight_of_log_fits = 0.53;

phi0 = [21.0, 3.6e-6, 0.048, 0.59, 0.29]

fit_orig = LsqFit.curve_fit((a,b)->model_Dmoves(a,b, in_log=data_in_log, data_ranges=dataset_ranges, w_log=weight_of_log_fits), 
                                data_x, scale_data(data_y, data_in_log, weight_of_log_fits), [phi0; 412]);

orig_cost=sum(scale_data(fit_orig.resid, data_in_log, weight_of_log_fits).^2)

par_results = zeros(5,100)
par_costs = zeros(1,100)
for i1 = 1:100
    fit_red = LsqFit.curve_fit((a,b)->model_LDmoves(a,b, in_log=data_in_log, data_ranges=dataset_ranges, w_log=weight_of_log_fits), 
    data_x, scale_data(data_y, data_in_log, weight_of_log_fits), [0.91; phi0[3:5]; 3]+randn(5,));
    
    par_results[:,i1] = fit_red.param
    par_costs[i1] = sum(scale_data(fit_red.resid, data_in_log, weight_of_log_fits).^2)
end

In [ ]:
@show par_costs
@show minimum(par_costs)
@show orig_cost

In [ ]:
y_pred_orig = model_Dmoves(data_x, [phi0, 412], data_ranges=dataset_ranges, in_log=data_in_log);
y_pred_origfit = model_Dmoves(data_x, fit_orig.param, data_ranges=dataset_ranges, in_log=data_in_log);
y_pred_redfit = model_LDmoves(data_x, par_results[:,sortperm(par_costs[:])[1]], data_ranges=dataset_ranges, in_log=data_in_log);

In [ ]:
using PlotlyJS


p1 = Plot([scatter(;x=data_x[dataset_ranges[3]], y=data_y[dataset_ranges[3]]),
    scatter(;x=data_x[dataset_ranges[4]], y=data_y[dataset_ranges[4]]),
    scatter(;x=data_x[dataset_ranges[3]], y=y_pred_origfit[dataset_ranges[3]]),
    scatter(;x=data_x[dataset_ranges[4]], y=y_pred_origfit[dataset_ranges[4]]),
    scatter(;x=data_x[dataset_ranges[3]], y=y_pred_redfit[dataset_ranges[3]]),
    scatter(;x=data_x[dataset_ranges[4]], y=y_pred_redfit[dataset_ranges[4]])
    ]
        )

p2 = Plot([scatter(;x=data_x[dataset_ranges[1]], y=data_y[dataset_ranges[1]]),
    scatter(;x=data_x[dataset_ranges[2]], y=data_y[dataset_ranges[2]]),
    scatter(;x=data_x[dataset_ranges[1]], y=y_pred_origfit[dataset_ranges[1]]),
    scatter(;x=data_x[dataset_ranges[2]], y=y_pred_origfit[dataset_ranges[2]]),
    scatter(;x=data_x[dataset_ranges[1]], y=y_pred_redfit[dataset_ranges[1]]),
    scatter(;x=data_x[dataset_ranges[2]], y=y_pred_redfit[dataset_ranges[2]])
    ]
        )

plt = plot([p1
    p2])


In [ ]:
PlotlyJS.relayout!(plt, height=1000)

In [ ]:
#@show fit_orig.param
#@show fit_red.param

# Model reduction while fitting log and classic data simultaneously

In [ ]:
# Enable all parameters to move between Control and LRRC26, see what gets reduced

function model_allmoves(x::AbstractArray, ϕ::AbstractArray; in_log=(zeros(size(x)).==1), data_ranges=Any[collect(1:length(x))], w_log=1)
    y_out1 = model_orig(x[data_ranges[1]], [ϕ[1]; ϕ[2:5]])
    y_out2 = model_orig(x[data_ranges[2]], [ϕ[6]; ϕ[7:10]])
    y_out3 = model_orig(x[data_ranges[3]], [ϕ[1]; ϕ[2:5]])
    y_out4 = model_orig(x[data_ranges[4]], [ϕ[6]; ϕ[7:10]])
    y_out = [y_out1; y_out2; y_out3; y_out4]
    return scale_data(to_log(y_out, in_log), in_log, w_log);
end

In [ ]:
# Find best fit point
weight_of_log_fits = w_log;

par_results = zeros(10,100)
par_costs = zeros(1,100)
for i1 = 1:100
    fit_orig = LsqFit.curve_fit((a,b)->model_allmoves(a,b, in_log=data_in_log, data_ranges=dataset_ranges, w_log=weight_of_log_fits), 
                                data_x, scale_data(data_y, data_in_log, weight_of_log_fits), 
                                [phi0, 412, phi0[2:5]].*2.*rand(10,));

    par_results[:,i1] = fit_orig.param
    par_costs[i1] = sum(scale_data(fit_orig.resid, data_in_log, weight_of_log_fits).^2)
end

y_pred_origfit = model_allmoves(data_x, par_results[:,sortperm(par_costs[:])[1]], data_ranges=dataset_ranges, in_log=data_in_log);



In [ ]:
@show minimum(par_costs)
@show par_results[:, sortperm(par_costs[:])[1]]

In [ ]:

p1 = Plot([scatter(;x=data_x[dataset_ranges[3]], y=data_y[dataset_ranges[3]]),
    scatter(;x=data_x[dataset_ranges[4]], y=data_y[dataset_ranges[4]]),
    scatter(;x=data_x[dataset_ranges[3]], y=y_pred_origfit[dataset_ranges[3]]),
    scatter(;x=data_x[dataset_ranges[4]], y=y_pred_origfit[dataset_ranges[4]]),
    ]
        )

p2 = Plot([scatter(;x=data_x[dataset_ranges[1]], y=data_y[dataset_ranges[1]]),
    scatter(;x=data_x[dataset_ranges[2]], y=data_y[dataset_ranges[2]]),
    scatter(;x=data_x[dataset_ranges[1]], y=y_pred_origfit[dataset_ranges[1]]),
    scatter(;x=data_x[dataset_ranges[2]], y=y_pred_origfit[dataset_ranges[2]]),
    ]
        )

plt = plot([p1
    p2])

In [ ]:
PlotlyJS.relayout!(plt, height=1000)

In [ ]:
#@show par_costs

In [ ]:
# Run MBAM on the complete model and see what happens

basic_model(param::AbstractArray, data::AbstractArray; model_id=0)=model_allmoves(data,param, in_log=data_in_log, data_ranges=dataset_ranges, w_log=weight_of_log_fits);

data_x_grid = [Real(d1) for d1 in data_x];

phi_joint0 = copy(par_results[:, sortperm(par_costs[:])[1]])
#= Code to run in normal julia (so that chrome doesnt die)

outp = MBAM(basic_model, phi_joint0, data_x_grid, model_iters=[0], 
move_dir=[-1], boundary_time=10, verbose=2, reduce_func=BK_reduce);

=#

# Figure 4c-d models for optimal information

In [1]:
using LsqFit
using Physical

elec_resp = (Volt*(-1)*e_electron/(k_boltzmann*304*Kelvin));


function bk_model_2010(ϕ::AbstractArray, x::AbstractArray; model_id=[])
    if model_id == 0 # Aldrich 2010, eq. 1
        D = ϕ[1]; L0 = ϕ[2]; J0 = ϕ[3]; zJ = ϕ[4]; zL = ϕ[5];
        Popen = (1+
        (1.+J0.*exp(-zJ .* x .* elec_resp )).^4
        ./
        ( L0.*exp(-zL .* x .* elec_resp ).*(1+D.*J0.*exp(-zJ .* x .* elec_resp )).^4)
        ).^-1

        return Popen
    end
    
    if model_id == 1
        DL = ϕ[1]; J0 = ϕ[2]; zJ = ϕ[3]; zL = ϕ[4];
        Popen = (1+
        (1.+J0.*exp(-zJ .* x .* elec_resp )).^4
        ./
        ( exp(-zL .* x .* elec_resp ).*(DL.*J0.*exp(-zJ .* x .* elec_resp )).^4)
        ).^-1

        return Popen
    end
    
    if model_id == 2
        DL = ϕ[1]; J0 = ϕ[2]; zJ = ϕ[3]
        Popen = (1+
        (1.+J0.*exp(-zJ .* x .* elec_resp )).^4
        ./
        ((DL.*J0.*exp(-zJ .* x .* elec_resp )).^4)
        ).^-1

        return Popen
    end
    
    ########################################################################
    ########################################################################
    ########################################################################
    # Aldrich 2010, eq. 1 in log space
    if model_id==10 
        D = ϕ[1]; L0 = ϕ[2]; J0 = ϕ[3]; zJ = ϕ[4]; zL = ϕ[5];
        Popen = (1+
        (1.+J0.*exp(-zJ .* x .* elec_resp )).^4
        ./
        ( L0.*exp(-zL .* x .* elec_resp ).*((1+D.*J0.*exp(-zJ .* x .* elec_resp )).^4))
        ).^-1

        logPopen = copy(Popen);
        for p1 = 1:length(logPopen)
            logPopen[p1] = Popen[p1]>0?log10(Popen[p1]):-10
        end
        return logPopen
    end
    
    if model_id==11
        DJ = ϕ[1]; L0 = ϕ[2]; zJ = ϕ[3]; zL = ϕ[4];
        Popen = (1+
        1.^4
        ./
        ( L0.*exp(-zL .* x .* elec_resp ).*((1+DJ.*exp(-zJ .* x .* elec_resp )).^4))
        ).^-1

        logPopen = copy(Popen);
        for p1 = 1:length(logPopen)
            logPopen[p1] = Popen[p1]>0?log10(Popen[p1]):-10
        end
        return logPopen
    end
    
    
    if model_id==12
        DJ = ϕ[1]; L0 = ϕ[2]; zJ = ϕ[3];
        Popen = (1+
        1.^4
        ./
        ( L0.*((1+DJ.*exp(-zJ .* x .* elec_resp )).^4))
        ).^-1

        logPopen = copy(Popen);
        for p1 = 1:length(logPopen)
            logPopen[p1] = Popen[p1]>0?log10(Popen[p1]):-10
        end
        return logPopen
    end
    
    ########################################################################
    ########################################################################
    ########################################################################
    # Aldrich 2010, eq. 1 in log space, but n = 17 (optimal inf. theory sense)
    if model_id==20
        D = ϕ[1]; L0 = ϕ[2]; J0 = ϕ[3]; zJ = ϕ[4]; zL = ϕ[5];
        Popen = (1+
        (1.+J0.*exp(-zJ .* x .* elec_resp )).^17
        ./
        ( L0.*exp(-zL .* x .* elec_resp ).*((1+D.*J0.*exp(-zJ .* x .* elec_resp )).^17))
        ).^-1

        logPopen = copy(Popen);
        for p1 = 1:length(logPopen)
            logPopen[p1] = Popen[p1]>0?log10(Popen[p1]):-10
        end
        return logPopen
    end
    
    if model_id==21
        DJ = ϕ[1]; L0 = ϕ[2]; zJ = ϕ[3]; zL = ϕ[4];
        Popen = (1+
        1.^17
        ./
        ( L0.*exp(-zL .* x .* elec_resp ).*((1+DJ.*exp(-zJ .* x .* elec_resp )).^17))
        ).^-1

        logPopen = copy(Popen);
        for p1 = 1:length(logPopen)
            logPopen[p1] = Popen[p1]>0?log10(Popen[p1]):-10
        end
        return logPopen
    end
    
    if model_id==22
        DJ = ϕ[1]; L0 = ϕ[2]; zJ = ϕ[3];
        Popen = (1+
        1.^17
        ./
        ( L0.*((1+DJ.*exp(-zJ .* x .* elec_resp )).^17))
        ).^-1

        logPopen = copy(Popen);
        for p1 = 1:length(logPopen)
            logPopen[p1] = Popen[p1]>0?log10(Popen[p1]):-10
        end
        return logPopen
    end
    
end

bk_model_2010 (generic function with 1 method)

In [2]:
function reduce_model(phi::AbstractArray; model_id=[], in_log=1)
    if in_log==1
        f_add = (a,x,b,y) -> a*x + b*y
    else
        f_add = (a,x,b,y) -> (x.^a)*(y.^b)
    end
    
    phi_red = deepcopy(phi)
    
    if model_id==0
        phi_red = zeros(size(phi)[1]-1)
        phi_red[1] = f_add(1, phi[1], 0.25, phi[2])
        phi_red[2] = phi[3]
        phi_red[3] = phi[4]
        phi_red[4] = phi[5]
    end
    
    if model_id == 1
        phi_red = zeros(size(phi)[1]-1)
        phi_red[1] = phi[1]
        phi_red[2] = phi[2]
        phi_red[3] = phi[3]
    end
    
    if model_id==10
        phi_red = zeros(size(phi)[1]-1)
        phi_red[1] = f_add(1, phi[1], 1, phi[3])
        phi_red[2] = phi[2]
        phi_red[3] = phi[4]
        phi_red[4] = phi[5]
    end
    
    if model_id == 11
        phi_red = zeros(size(phi)[1]-1)
        phi_red[1] = phi[1]
        phi_red[2] = phi[2]
        phi_red[3] = phi[3]
    end
    
    if model_id==20
        phi_red = zeros(size(phi)[1]-1)
        phi_red[1] = f_add(1, phi[1], 1, phi[3])
        phi_red[2] = phi[2]
        phi_red[3] = phi[4]
        phi_red[4] = phi[5]
    end
    
    if model_id == 21
        phi_red = zeros(size(phi)[1]-1)
        phi_red[1] = phi[1]
        phi_red[2] = phi[2]
        phi_red[3] = phi[3]
    end
    
    return phi_red
end

reduce_model (generic function with 1 method)

In [3]:
using NBInclude #Includes for ipynb-s
#using JLD # Julia data I/O
nbinclude("MBAM.ipynb")

MBAM (generic function with 1 method)

In [4]:
data_x_full = collect(-160:20:200)*1e-3
data_x_low = collect(-160:20:80)*1e-3
data_x_high = collect(80:20:200)*1e-3;

In [5]:
phi0 = [21, 3.6e-6, 0.048, 0.59, 0.29];

In [6]:
outp_high = MBAM(bk_model_2010, phi0, data_x_high, model_iters=[0, 1, 2], move_dir=[1, -1, -1],
reduce_func=reduce_model, verbose=1);

  0.419009 seconds (395.78 k allocations: 18.233 MB, 1.35% gc time)
Model iteration 0, Single g_cost call takes: 
  0.110286 seconds (112.76 k allocations: 5.028 MB)
Model iteration 0, initial values:
      D = [  1.94e-08,   1.01e-05,   1.29e-03,   1.05e-01,   7.89e+00]
   cost = 1.718892475365608e-32
phi_cur = [      3.04,     -12.53,      -3.04,      -0.53,      -1.24]
      v = [  3.05e-01,  -9.47e-01,   3.38e-02,  -2.60e-02,  -9.14e-02]


Model iteration 0, final values:
      D = [  4.53e-13,   3.01e-06,   1.92e-03,   6.34e-02,   9.13e+00]
   cost = 2.5250260928111526e-7
phi_cur = [      8.41,     -32.80,      -2.97,      -0.62,      -1.73]
      v = [  2.43e-01,  -9.70e-01,   1.24e-05,  -1.23e-04,  -9.92e-04]


Model iteration 1, Single g_cost call takes: 
  0.002927 seconds (673 allocations: 36.602 KB)
Model iteration 1, initial values:
      D = [  2.98e-06,   1.91e-03,   6.20e-02,   8.94e+00]
   cost = 3.5065176005451135e-7
phi_cur = [      0.21,      -2.97,      -0.62,      

In [20]:
outp_low = MBAM(bk_model_2010, phi0, data_x_low, model_iters=[10, 11, 12], move_dir = [1, 1, 1], 
reduce_func=reduce_model, verbose=1);

  0.000049 seconds (103 allocations: 6.203 KB)
Model iteration 10, Single g_cost call takes: 
  0.000029 seconds (111 allocations: 6.688 KB)
Model iteration 10, initial values:
      D = [  6.06e-04,   1.34e-02,   1.02e+00,   5.59e+00,   2.05e+01]
   cost = 1.1832913578315177e-30
phi_cur = [      3.04,     -12.53,      -3.04,      -0.53,      -1.24]
      v = [  7.05e-01,  -3.72e-01,  -5.65e-01,  -9.28e-02,  -1.93e-01]


Model iteration 10, final values:
      D = [  1.02e-09,   3.74e-03,   5.65e-01,   7.15e+00,   2.61e+01]
   cost = 0.0014721721400098161
phi_cur = [      9.94,     -13.27,      -9.66,      -0.67,      -1.70]
      v = [  7.07e-01,  -7.92e-04,  -7.07e-01,  -1.20e-04,  -5.99e-04]


Model iteration 11, Single g_cost call takes: 
  0.000042 seconds (100 allocations: 5.422 KB)
Model iteration 11, initial values:
      D = [  3.54e-03,   5.20e-01,   6.17e+00,   1.74e+01]
   cost = 0.001476466516934073
phi_cur = [      0.27,     -13.27,      -0.67,      -1.70]
      v = [  3.

In [10]:
outp_low17 = MBAM(bk_model_2010, phi0, data_x_low, model_iters=[20, 21, 22], move_dir=[1, -1, 1],
reduce_func=reduce_model, verbose=1);

  0.000050 seconds (103 allocations: 6.203 KB)
Model iteration 20, Single g_cost call takes: 
  0.000029 seconds (111 allocations: 6.688 KB)
Model iteration 20, initial values:
      D = [  1.86e-05,   1.51e-03,   7.66e-01,   8.67e+00,   5.95e+01]
   cost = 1.232595164407831e-32
phi_cur = [      3.04,     -12.53,      -3.04,      -0.53,      -1.24]
      v = [  6.64e-01,  -2.40e-01,  -6.99e-01,  -3.37e-02,  -1.08e-01]


Model iteration 20, final values:
      D = [  1.82e-10,   1.06e-03,   5.57e-01,   8.92e+00,   6.30e+01]
   cost = 3.685084805047665e-5
phi_cur = [      8.75,     -12.90,      -8.79,      -0.58,      -1.42]
      v = [  7.07e-01,  -8.80e-04,  -7.07e-01,  -1.16e-04,  -4.61e-04]


Model iteration 21, Single g_cost call takes: 
  0.000034 seconds (100 allocations: 5.422 KB)
Model iteration 21, initial values:
      D = [  1.06e-03,   5.56e-01,   7.25e+00,   3.90e+01]
   cost = 3.894792338247559e-5
phi_cur = [     -0.04,     -12.90,      -0.58,      -1.42]
      v = [ -9.86

In [29]:
using PlotlyJS
plot([Plot(scatter(;x=data_x_full*1e3, y=bk_model_2010(exp(outp_low17[1][1,1]), data_x_full, model_id=20))) 
    Plot(scatter(;x=data_x_full*1e3, y=bk_model_2010(exp(outp_low17[1][3,2]), data_x_full, model_id=22)))])

In [27]:
plot([Plot(scatter(;x=data_x_full*1e3, y=bk_model_2010(exp(outp_low[1][1,1]), data_x_full, model_id=10))) 
    Plot(scatter(;x=data_x_full*1e3, y=bk_model_2010(exp(outp_low[1][3,2]), data_x_full, model_id=12)))])